In [1]:
!pip install wget

  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9656 sha256=5846614fccaa6cf80cb6663ac77e9442a8289bea1914f67e662055a96e0a3f73
  Stored in directory: /root/.cache/pip/wheels/8b/f1/7f/5c94f0a7a505ca1c81cd1d9208ae2064675d97582078e6c769
Successfully built wget


In [33]:
import gensim
import numpy as np
import pandas as pd
from sklearn import metrics
from gensim.corpora import Dictionary
import wget
import urllib.request
import pickle
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *

In [15]:
# load dictionary

url = 'https://raw.githubusercontent.com/namvnvx/DS107_Project/main/dictionary.gensim'
wget.download(url, 'dictionary.gensim')
dictionary = Dictionary.load('dictionary.gensim')

# load 3 corpus từng sentiment

url1_1 = 'https://raw.githubusercontent.com/namvnvx/DS107_Project/main/Corpus/positive_corpus.pkl'
with urllib.request.urlopen(url1_1) as e:
    bow_pos = pickle.load(e)

url1_2 = 'https://raw.githubusercontent.com/namvnvx/DS107_Project/main/Corpus/neutral_corpus.pkl'
with urllib.request.urlopen(url1_2) as f:
    bow_neu = pickle.load(f)

url1_3 = 'https://raw.githubusercontent.com/namvnvx/DS107_Project/main/Corpus/negative_corpus.pkl'
with urllib.request.urlopen(url1_3) as g:
    bow_neg = pickle.load(g)

# load test_data

test_data = pd.read_csv('https://raw.githubusercontent.com/namvnvx/DS107_Project/main/Demo_data/test_data.csv')
X_test = test_data['tweet'].values
y_test = test_data['class'].values

In [50]:
lda_neu =  gensim.models.LdaMulticore(bow_neu, num_topics = 10, id2word = dictionary, passes = 10, workers = 3)
lda_pos =  gensim.models.LdaMulticore(bow_pos, num_topics = 10, id2word = dictionary, passes = 10, workers = 3)
lda_neg =  gensim.models.LdaMulticore(bow_neg, num_topics = 10, id2word = dictionary, passes = 10, workers = 3)

In [37]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [38]:

stemmer = SnowballStemmer("english")
def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))

# Tokenize and lemmatize
def pre_process(data):
    final = []
    for text in data:
      result=[]
      for token in gensim.utils.simple_preprocess(text) :
          if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
              result.append(lemmatize_stemming(token))
      final.append(result)

    return final

In [51]:
class_0 = []
class_1 = []
class_2 = []

X_test_processing = pre_process(X_test)

for text in X_test_processing:
  bow_vector =  dictionary.doc2bow(text)
    #Train neutral topic
  neu_topic = lda_neu.get_document_topics(bow_vector)
  neu_score = sorted(neu_topic, key=lambda x: -x[1])[0][1]
  class_1.append(neu_score)

    #Train positive topic
  pos_topic = lda_pos.get_document_topics(bow_vector)
  pos_score = sorted(pos_topic, key=lambda x: -x[1])[0][1]
  class_2.append(pos_score)

    #Train negative topic
  neg_topic = lda_neg.get_document_topics(bow_vector)
  neg_score = sorted(neg_topic, key=lambda x: -x[1])[0][1]
  class_0.append(neg_score)

  scores_array = np.array([neu_score, pos_score, neg_score])
  max_score = np.max(scores_array)

In [43]:
max(class_0[3], class_1[3], class_2[3])

0.51256984

In [44]:
class_pred = []
# 1 phương thức đánh giá nào đó
for neg, neu, pos in zip(class_0, class_1, class_2):
  m = max(neg, neu, pos)
  if m == neg:
    class_pred.append(0)
    continue
  elif m == neu:
    class_pred.append(1)
    continue
  elif m == pos:
    class_pred.append(2)


In [45]:
accuracy = metrics.accuracy_score(class_pred, y_test)
accuracy

0.3846046576488635

In [27]:
from sklearn.metrics import f1_score
f1 = f1_score(class_pred, y_test,average=None)
print("F1 Score:", f1)

F1 Score: [0.37533221 0.38234402 0.40045946]


In [49]:
for idx, topic in lda_neg.print_topics(-1):
    print("Topic: {} \nWords: {}".format(idx, topic ))
    print("\n")


Topic: 0 
Words: 0.054*"peopl" + 0.020*"coronaviru" + 0.018*"patient" + 0.017*"child" + 0.012*"like" + 0.012*"open" + 0.012*"think" + 0.011*"time" + 0.009*"someon" + 0.008*"lockdown"


Topic: 1 
Words: 0.035*"home" + 0.026*"nur" + 0.018*"anoth" + 0.018*"care" + 0.012*"govern" + 0.012*"dead" + 0.010*"death" + 0.010*"cuomo" + 0.010*"understand" + 0.009*"worker"


Topic: 2 
Words: 0.046*"fake" + 0.019*"death" + 0.019*"pandem" + 0.014*"year" + 0.014*"lockdown" + 0.010*"danger" + 0.009*"measur" + 0.009*"medic" + 0.009*"work" + 0.008*"news"


Topic: 3 
Words: 0.023*"work" + 0.018*"coronaviru" + 0.017*"peopl" + 0.014*"happen" + 0.012*"social" + 0.011*"risk" + 0.009*"death" + 0.009*"start" + 0.009*"busi" + 0.009*"week"


Topic: 4 
Words: 0.019*"peopl" + 0.017*"work" + 0.016*"stress" + 0.015*"know" + 0.008*"famili" + 0.008*"health" + 0.008*"hospit" + 0.008*"kill" + 0.008*"everi" + 0.007*"crisi"


Topic: 5 
Words: 0.060*"death" + 0.043*"peopl" + 0.033*"test" + 0.024*"coronaviru" + 0.020*"countri